In [55]:
import tensorflow as tf
import os
import numpy as np

In [56]:
base_dir=r"/content/drive/MyDrive/boxs_images_dataset"

In [57]:
base_dir

'/content/drive/MyDrive/boxs_images_dataset'

In [58]:
IMAGE_SIZE=224
BATCH_SIZE=64

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1)

validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

In [59]:
train_genarator=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
    ) 

validation_generator=validation_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
    )

Found 28 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


In [60]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob

In [61]:
IMAGE_SIZE=[224,224]
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [62]:
for layer in vgg.layers:
    layer.trainable=False

In [63]:
folders=glob(r"/content/drive/MyDrive/boxs_images_dataset/*")
print(len(folders))

2


In [64]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [65]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
epoch=10

history=model.fit(train_genarator,
                  steps_per_epoch=len(train_genarator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                 )

Epoch 1/10
1/1 [==============================] - 15s 15s/step - loss: 0.1956 - accuracy: 0.8929 - val_loss: 0.2774 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 15s 15s/step - loss: 0.2265 - accuracy: 0.8214 - val_loss: 0.2942 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 15s 15s/step - loss: 0.1671 - accuracy: 0.9286 - val_loss: 0.3624 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 15s 15s/step - loss: 0.1846 - accuracy: 0.8929 - val_loss: 0.4938 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 15s 15s/step - loss: 0.1277 - accuracy: 0.9286 - val_loss: 0.6733 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 15s 15s/step - loss: 0.1314 - accuracy: 0.9286 - val_loss: 0.8880 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 15s 15s/step - loss: 0.1000 - accuracy: 1.0000 - val_loss: 1.0830 - val_accuracy: 0.5000
Epoch 8/10
1/1 [===========

In [73]:
from keras.preprocessing import image
import numpy as np

# img_pred=image.load_img(r"/content/drive/MyDrive/boxs_images_dataset/white_label/7000028242.JPG",target_size=(224,224))
img_pred=image.load_img(r"/content/drive/MyDrive/boxs_images_dataset/cardboard_label/7000028339 b.JPG",target_size=(224,224))
img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)

rslt= model.predict(img_pred)
print(rslt)
if rslt[0][0]>rslt[0][1]:
    prediction="cardboard"
else:
    prediction="white_label"
print(prediction)

[[1. 0.]]
cardboard
